
## Imports and/or google colaboratory setup

In [ ]:
github_repo_name = 'bandits'
github_url = 'https://github.com/matanster/' + github_repo_name

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import random

if get_ipython().__class__.__module__ == "google.colab._shell":
    print('running in google colab')
    !git clone $github_url # clone the repo so that the python files of the repo are available to the runtime
    exec('import ' + github_repo_name + '.python_lib.bandit as bd')
    exec('from ' + github_repo_name + '.python_lib.util import unique_argmax')
    exec('from ' + github_repo_name + '.python_lib.plot import interval_binner, plot_series, plot_serie')


else:
    print('running locally')
    
    import python_lib.bandit as bd
    from python_lib.util import unique_argmax
    from python_lib.plot import interval_binner, plot_series, plot_serie
    
    import ipycache # consult https://github.com/rossant/ipycache#usage for usage
    %load_ext ipycache 
    cache_file = 'cache-file' 






## a Bernoulli MAB "solved" with thompson sampling
bandits (the true world) come from library code;
solution code is in the notebook itself flexible adaptation. <br>
inspiration credited to https://peterroelants.github.io/posts/multi-armed-bandit-implementation/

In [ ]:
def thompson_select(beta_priors, bandits):   
    ''' samples from the prior distribution per bandit, and selects the argmax bandit '''

    beta_dist_samples = list(map(lambda k: np.random.beta(*beta_priors[k], size=1)[0], range(bandits.num_of_bandits)))
    selection = np.argmax(beta_dist_samples)
    
    if selection.size == 1:
        return selection.flat[0]
    else:
        return random.choice(selection) # tie break the argmax by random
    
    

def simulate(bandits, beta_priors, n):
    ''' unleash online learning over n turns, keeping a record of every turn'''
    
    beta_priors = beta_priors.copy()
    
    round_records = []

    for i in range(n):

        round_record = dict(priors=list(beta_priors))

        k = thompson_select(beta_priors, bandits)
        outcome = bandits.pull(k)

        # the posterior computation here is a specific case to outcomes of 0 and 1. 
        # https://www.youtube.com/watch?v=hKYvZF9wXkk discusses the general case
        if outcome == 1:
            beta_priors[k] = (beta_priors[k][0]+1, beta_priors[k][1]) # increment the alpha param
        elif outcome == 0:
            beta_priors[k] = (beta_priors[k][0], beta_priors[k][1]+1) # increment the beta param
        else:
            raise ValueError()

        round_record.update(dict(k=k, outcome=outcome, posteriors=list(beta_priors)))

        round_records.append(round_record)
        
    return pd.DataFrame.from_records(round_records)

In [ ]:
def simulate_n_times(bandits, beta_priors, turns, simulations):
    ''' run the same simulation m times, accumulating a record of every simulation's every turn '''
    simulation_dfs = []
    for i in range(simulations):
        simulation_df = simulate(bandits, beta_priors, turns)
        simulation_dfs.append(simulation_df)
    return simulation_dfs

In [ ]:
bandits     = bd.BanditsVec(list(map(bd.Bernoulli, [0.03, 0.3, 0.4, 0.8, 0.9, 0.95])), bd.Bernoulli.best_bandit)
beta_priors = list(map(lambda bandit: (1,1), range(bandits.num_of_bandits)))

In [ ]:
def compute_cross_simulation_stats(simulation_dfs, columns, stat_specs):
    ''' computes turn-wise stats across simulation dataframes.
        this is like building a data cube and aggregating the desired statistics on its new axis. '''
    
    # an intermediary dataframe where the index is the rounds, and each cell is the array 
    # of values across all simulations, for each original column
    rounds_df = pd.DataFrame(index=simulation_dfs[0].index, columns=columns)
    for column in columns:          
        for round in range(simulation_dfs[0].shape[0]):
            round_values = [simulation_df.iloc[round][column] for simulation_df in simulation_dfs]
            rounds_df[column].iloc[round] = round_values

    # the result dataframe where the index is the rounds, there is a column for every column+stat combo,
    # and the values are the implied statistics
    rounds_stats_df = pd.DataFrame(index=rounds_df.index, columns=[column + '-' + stat_spec['name'] for column in columns for stat_spec in stat_specs])
    for round in range(simulation_dfs[0].shape[0]):
        for column in columns:          
            for stat_spec in stat_specs:
                rounds_stats_df.iloc[round][column + '-' + stat_spec['name']] = \
                    stat_spec['fn'](rounds_df.iloc[round][column])
        
    return rounds_stats_df


def get_cross_simulation_stats(simulation_dfs):
    ''' computes the average and std per turn, for items of interest '''
    return compute_cross_simulation_stats(simulation_dfs, ['k', 'outcome'], [dict(name='avg', fn=np.mean), dict(name='std', fn=np.std)])

In [ ]:
# %%cache $cache_file simulation_dfs, round_stats
simulation_dfs = simulate_n_times(bandits, beta_priors, 200,1000)
round_stats = get_cross_simulation_stats(simulation_dfs)

In [ ]:
plot_series('reward per turn over {} simulations of {} rounds each'.format(simulation_dfs[0].shape[0], len(simulation_dfs)),
            'turn', 'reward',            
            round_stats,
            [dict(column='outcome-avg', display_name='average'), dict(column='outcome-std', display_name='std')], 
            lines=False,
            marker_size=3)

In [ ]:
plot_series('arm selection per turn over {} simulations of {} rounds each'.format(simulation_dfs[0].shape[0], len(simulation_dfs)),
            'turn', 'arm',            
            round_stats,
            [dict(column='k-avg', display_name='average arm'), dict(column='k-std', display_name='std')], 
            lines=False,
            marker_size=3)

## Looking at the final posterior distributions
more precisely the distributions of the means of the posterior of each arm. 
<br> equally put, we take the mean of the alpha and beta per arm over all simulations, and plot those beta distributions.

In [ ]:
final_alpha_posteriors = pd.DataFrame(
    index=range(len(simulation_dfs)),
    columns = [arm for arm in range(bandits.num_of_bandits)])

final_beta_posteriors = pd.DataFrame(
    index=range(len(simulation_dfs)),
    columns = [arm for arm in range(bandits.num_of_bandits)])
    
for i, simulation_df in enumerate(simulation_dfs):
    for arm in range(bandits.num_of_bandits):
        final_alpha_posteriors.iloc[i][arm] = simulation_df.tail(1).iloc[0]['posteriors'][arm][0]
        final_beta_posteriors.iloc[i][arm]  = simulation_df.tail(1).iloc[0]['posteriors'][arm][1]
        
mean_alpha_posteriors = final_alpha_posteriors.mean()
mean_beta_posteriors  = final_beta_posteriors.mean()

In [ ]:
x = np.linspace(0,1,1000)
arm_estimates_df = pd.DataFrame(index=x)
for arm in range(bandits.num_of_bandits):   
    beta_distribution = scipy.stats.beta(mean_alpha_posteriors[arm], mean_beta_posteriors[arm])
    arm_estimates_df[arm] = [beta_distribution.pdf(i) for i in x]

In [ ]:
params_df = pd.DataFrame(index = pd.Index(range(bandits.num_of_bandits), name='arm'))
params_df['true param'] = pd.Series(map(lambda bandit: bandit.success_prob, bandits.bandits))
params_df['mean(α)']    = pd.Series(mean_alpha_posteriors)
params_df['mean(β)']    = pd.Series(mean_beta_posteriors)
params_df['std(α)']     = pd.Series(final_alpha_posteriors.std())
params_df['std(β)']     = pd.Series(final_beta_posteriors.std())
display(params_df)

In [ ]:
plot_series('final estimation of the probability density function per arm,<br>(averaged over all simulations)', '', '',
            arm_estimates_df, list(range(bandits.num_of_bandits)), 
            lines=True, fill=True, marker_size=1)

## TODO:

+ optimize the simulation functions by using numpy directly or otherwise
+ normalize all beta distributions into a probability distribution (trivial)
+ fancy: make a dash page/app that chooses distibutions more dynamically from the simulations cube